In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./file/2.make.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Year'] = input_['Year'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        try:
            data = {'cssClass': 'form-control',
                    'dropdownType': '2',
                    'selectedValue': '',
                    'vehicleAttributes': {'Make': f'''{input_.loc[a, 'Make']}''',
                                          'MakeID': f'''{input_.loc[a, 'Make Code']}''',
                                          'ModelID': 0,
                                          'SkinID': '1',
                                          'Year': f'''{input_.loc[a, 'Year']}'''}}
            
            b = 0
            while True:
                b += 1
                
                try:
                    resp = requests.post('https://www.dormanproducts.com/ApplicationGuideService.asmx/BuildDropdown',
                                         data=json.dumps(data),
                                         headers=get_header(**{'Content-Type': 'application/json; charset=UTF-8'}),
                                         proxies=get_proxy(),
                                         timeout=(10, 10))

                    if resp.status_code == 200:
                        break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            json_ = resp.json()

            # = = = = = = = = = = = = = = =

            soup = BeautifulSoup(json_['d'], 'lxml')
            html = etree.HTML(str(soup))

            # = = = = = = = = = = = = = = =

            list_model_code = html.xpath('//select[@id="modelDropdown"]/option/@value')[1:]

            # = = = = = = = = = = = = = = =

            list_model = html.xpath('//select[@id="modelDropdown"]/option/text()')[1:]

            # = = = = = = = = = = = = = = =

            if not list_model:
                raise

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Make': input_.loc[a, 'Make'],
                                    'Make Code': input_.loc[a, 'Make Code'],
                                    'Model': list_model,
                                    'Model Code': list_model_code,
                                    'Year': input_.loc[a, 'Year']})
            
            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {input_.loc[a, 'Make']} > {input_.loc[a, 'Year']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Make', 'Model', 'Year'],
                                                ascending=[True, True, False],
                                                ignore_index=True)
    output_correct.to_excel(f'''./file/3.model-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Make', 'Year'],
                                            ascending=[True, False],
                                            ignore_index=True)
    output_error.to_excel(f'''./file/model_error.xlsx''', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

[状态：ok，尝试次数：7] - Renegade > 2014
[剩余数量：1000] - [当前时间：18:57:57]

[状态：ok，尝试次数：10] - Renegade > 2007
[剩余数量：999] - [当前时间：18:57:57]

[状态：ok，尝试次数：17] - Renegade > 2013
[剩余数量：998] - [当前时间：18:57:58]

[状态：ok，尝试次数：8] - Renegade > 2006
[剩余数量：997] - [当前时间：18:57:58]

[状态：ok，尝试次数：6] - Renegade > 2023
[剩余数量：996] - [当前时间：18:57:58]

[状态：ok，尝试次数：1] - Roadmaster Rail > 2012
[剩余数量：995] - [当前时间：18:57:58]

[状态：ok，尝试次数：23] - Renegade > 2010
[剩余数量：994] - [当前时间：18:57:58]

[状态：ok，尝试次数：3] - Renault > 1975
[剩余数量：993] - [当前时间：18:57:58]

[状态：ok，尝试次数：75] - Renault > 2023
[剩余数量：992] - [当前时间：18:57:58]

[状态：ok，尝试次数：23] - Renault > 2001
[剩余数量：991] - [当前时间：18:57:58]

[状态：ok，尝试次数：1] - Roadmaster Rail > 2004
[剩余数量：990] - [当前时间：18:57:58]

[状态：ok，尝试次数：8] - Renegade > 2005
[剩余数量：989] - [当前时间：18:57:58]

[状态：ok，尝试次数：17] - Renault > 1984
[剩余数量：988] - [当前时间：18:57:59]

[状态：ok，尝试次数：32] - Renault > 1971
[剩余数量：987] - [当前时间：18:57:59]

[状态：ok，尝试次数：18] - Renault > 1977
[剩余数量：986] - [当前时间：18:57:59]

[状态：ok，尝试次数：7] - Roadmaster Rail > 200